In [1]:
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import numpy as np
import sys

sys.path.append(r'P:\1000545-054-globalbeaches\15_GlobalCoastalAtlas\global-coastal-atlas\notebooks\01. utils')

from plot_funcs import future_shorelinepositions


In [2]:
# load in and keep only sandy transects
df_annual = pd.read_csv('00. data/westeurope_annual.csv')
df_annual = df_annual[df_annual['flag_sandy'] == True]


df_annual = gpd.GeoDataFrame(df_annual, geometry= gpd.points_from_xy(df_annual.Intersect_lon, df_annual.Intersect_lat)).set_crs('epsg:3587')
df_annual = df_annual.to_crs('+proj=wintri')

#### GHG emmisions (Vousdoukas, 2021)

In [3]:
basepath_vous = r'P:\1000545-054-globalbeaches\01_Global_Future_Shorelines\globalErosionProjectionsDataset_linear'

#RCP4.5 2050
df_vous_45_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2050.csv'), header= None)
df_vous_45_2050 = df_vous_45_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2050', 3: 'p5_45_2050', 
                                                   4: 'p17_45_2050', 5: 'p50_45_2050', 6: 'p83_45_2050', 7: 
                                                   'p95_45_2050', 8: 'p99_45_2050'})
df_vous_45_2050 = df_vous_45_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP4.5 2100
df_vous_45_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP45_2100.csv'), header= None)
df_vous_45_2100 = df_vous_45_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_45_2100', 3: 'p5_45_2100', 
                                                   4: 'p17_45_2100', 5: 'p50_45_2100', 6: 'p83_45_2100', 7: 
                                                   'p95_45_2100', 8: 'p99_45_2100'})
df_vous_45_2100 = df_vous_45_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP8.5 2050
df_vous_85_2050 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2050.csv'), header= None)
df_vous_85_2050 = df_vous_85_2050.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2050', 3: 'p5_85_2050', 
                                                   4: 'p17_85_2050', 5: 'p50_85_2050', 6: 'p83_85_2050', 7: 
                                                   'p95_85_2050', 8: 'p99_85_2050'})
df_vous_85_2050 = df_vous_85_2050.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

#RCP8.5 2100
df_vous_85_2100 = pd.read_csv(os.path.join(basepath_vous, 'globalErosionProjections_SLR_Retreat_RCP85_2100.csv'), header= None)
df_vous_85_2100 = df_vous_85_2100.rename(columns= {0: 'latitude', 1: 'longitude', 2: 'p1_85_2100', 3: 'p5_85_2100', 
                                                   4: 'p17_85_2100', 5: 'p50_85_2100', 6: 'p83_85_2100', 7: 
                                                   'p95_85_2100', 8: 'p99_85_2100'})
df_vous_85_2100 = df_vous_85_2100.drop_duplicates(subset= ['latitude', 'longitude'], keep = 'first')

df_vous_45_2050.head(2)

,latitude,longitude,p1_45_2050,p5_45_2050,p17_45_2050,p50_45_2050,p83_45_2050,p95_45_2050,p99_45_2050
0,-50.378,-74.386,-2.0775,-1.6823,-1.2463,-0.744,-0.33039,-0.18465,-0.13225
1,-50.386,-74.303,-2.0773,-1.6899,-1.2751,-0.744,-0.34310,-0.18249,-0.12239


In [4]:
[set(df_vous_45_2050['latitude']).difference(x) for x in [set(df_vous_45_2100['latitude']), set(df_vous_85_2050['latitude']), set(df_vous_85_2100['latitude'])]]


[set(), set(), set()]

In [5]:
dfs_vous = [df_vous_45_2100, df_vous_85_2050, df_vous_85_2100]

df_vous = df_vous_45_2050.copy()
for dfv in dfs_vous:
    df_vous = df_vous.merge(dfv, on = ['latitude', 'longitude'], how = 'left')

assert len(df_vous_45_2050) == len(df_vous)

In [6]:
gdf_vous = gpd.GeoDataFrame(df_vous, geometry= gpd.points_from_xy(df_vous.longitude, df_vous_85_2100.latitude)).set_crs('epsg:3587')
gdf_vous = gdf_vous.to_crs('+proj=wintri')

##### Spatial Join on nearest transect

In [7]:
gdf_vous_join = df_annual.sjoin_nearest(gdf_vous, how = 'left', max_distance = 1000)
gdf_vous_join.head(2)


,Unnamed: 0,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt_annual,...,p83_85_2050,p95_85_2050,p99_85_2050,p1_85_2100,p5_85_2100,p17_85_2100,p50_85_2100,p83_85_2100,p95_85_2100,p99_85_2100
0,0,BOX_080_003_0,FRA,Europe,France,-91.82,4.547473e-14,True,2.0,"[19.0, 24.0]",...,-0.47418,-0.18553,-0.086275,-14.443,-10.65,-6.0675,-3.4541,-1.4868,-0.59291,-0.37737
11,11,BOX_080_003_11,FRA,Europe,France,-0.09,5.811708e-01,True,22.0,"[2.0, 15.0, 16.0, 17.0, 18.0, 19.0, 21.0, 22.0...",...,-0.47796,-0.15489,-0.079392,-14.371,-10.54,-6.2095,-3.4541,-1.4188,-0.56968,-0.29734


In [8]:
def future_timeseries(a, b, cor_2050, cor_2100):
    a = 0
    dt = np.arange(1984, 2101, 1)

    y2021 = a + (2021 - 1984) * b
    y2050 = a + (2050 - 1984) * b + cor_2050
    y2100 = a + (2100 - 1984) * b + cor_2100

    s = pd.Series(index = dt, dtype='float64')
    s.loc[1984] = a
    s.loc[2021] = y2021
    s.loc[2050] = y2050
    s.loc[2100] = y2100
    s = s.interpolate(method = 'index').values

    return s

In [9]:
# AMBIENT
gdf_vous_join['Ambient'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, 0, 0), axis = 1)

# RCP4.5
gdf_vous_join['RCP4.5_p5'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p5_45_2050, row.p5_45_2100), axis = 1)
gdf_vous_join['RCP4.5_p50'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p50_45_2050, row.p50_45_2100), axis = 1)
gdf_vous_join['RCP4.5_p95'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p95_45_2050, row.p95_45_2100), axis = 1)

# RCP8.5
gdf_vous_join['RCP8.5_p5'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p5_85_2050, row.p5_85_2100), axis = 1)
gdf_vous_join['RCP8.5_p50'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p50_85_2050, row.p50_85_2100), axis = 1)
gdf_vous_join['RCP8.5_p95'] = gdf_vous_join.apply(lambda row: future_timeseries(row.intercept, row.changerate, row.p95_85_2050, row.p95_85_2100), axis = 1)


In [10]:
gdf_vous_join.head(2)

,Unnamed: 0,transect_id,country_id,continent,country_name,changerate,changerate_unc,flag_sandy,no_shorelines,dt_annual,...,p83_85_2100,p95_85_2100,p99_85_2100,Ambient,RCP4.5_p5,RCP4.5_p50,RCP4.5_p95,RCP8.5_p5,RCP8.5_p50,RCP8.5_p95
0,0,BOX_080_003_0,FRA,Europe,France,-91.82,4.547473e-14,True,2.0,"[19.0, 24.0]",...,-1.4868,-0.59291,-0.37737,"[0.0, -91.82, -183.64, -275.46, -367.28, -459....","[0.0, -91.82, -183.64, -275.46, -367.28, -459....","[0.0, -91.82, -183.64, -275.46, -367.28, -459....","[0.0, -91.82, -183.64, -275.46, -367.28, -459....","[0.0, -91.82, -183.64, -275.46, -367.28, -459....","[0.0, -91.82, -183.64, -275.46, -367.28, -459....","[0.0, -91.82, -183.64, -275.46, -367.28, -459...."
11,11,BOX_080_003_11,FRA,Europe,France,-0.09,5.811708e-01,True,22.0,"[2.0, 15.0, 16.0, 17.0, 18.0, 19.0, 21.0, 22.0...",...,-1.4188,-0.56968,-0.29734,"[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999...","[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999...","[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999...","[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999...","[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999...","[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999...","[0.0, -0.09, -0.18, -0.27, -0.36, -0.449999999..."


In [11]:
box = 'BOX_187_084_11'
df = gdf_vous_join

future_shorelinepositions(df = df, box = box)

In [12]:
box = 'BOX_187_084_11'
df = gdf_vous_join

future_shorelinepositions(df = df, box = box)

In [14]:
box = 'BOX_187_090_49'
df = gdf_vous_join

future_shorelinepositions(df = df, box = box)